In [1]:
import yaml
import json
import os
from pathlib import Path

def read_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

def read_ipynb(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        notebook = json.load(file)
    
    markdown_content = []
    for cell in notebook['cells']:
        if cell['cell_type'] == 'markdown':
            markdown_content.extend(cell['source'])
        elif cell['cell_type'] == 'code':
            markdown_content.append('```python\n')
            markdown_content.extend(cell['source'])
            markdown_content.append('\n```\n')
    
    return ''.join(markdown_content)

def process_chapters(chapters, base_path):
    content = []
    for chapter in chapters:
        if 'file' in chapter:
            file_path = Path(base_path) / f"{chapter['file']}.ipynb"
            if file_path.exists():
                content.append(f"# {chapter.get('title', chapter['file'])}\n\n")
                content.append(read_ipynb(file_path))
            else:
                content.append(f"# {chapter.get('title', chapter['file'])}\n\n")
                content.append(f"File not found: {file_path}\n\n")
        if 'chapters' in chapter:
            content.extend(process_chapters(chapter['chapters'], base_path))
    return content

def toc_to_markdown(toc_path):
    toc_data = read_yaml(toc_path)
    base_path = Path(toc_path).parent
    
    content = [f"# {toc_data.get('root', 'Documentation')}\n\n"]
    
    for part in toc_data.get('parts', []):
        content.append(f"## {part['caption']}\n\n")
        content.extend(process_chapters(part.get('chapters', []), base_path))
    
    return '\n'.join(content)

In [3]:
toc_path = 'docs/_toc.yml'  # Update this path if your _toc.yml is located elsewhere
output_path = 'docs/combined_documentation.md'

markdown_content = toc_to_markdown(toc_path)

with open(output_path, 'w', encoding='utf-8') as file:
    file.write(markdown_content)

print(f"Combined Markdown file has been created: {output_path}")

Combined Markdown file has been created: docs/combined_documentation.md
